In [3]:
from __future__ import division
import os
import sys
import copy
import numpy as np
from numpy import ma
import math as m
import pandas as pd
import matplotlib.patches as mpatches
import string
import cmasher as cmr
import docx

import pickle
from numpy import genfromtxt
from scipy.interpolate import griddata
from matplotlib import rc
import matplotlib.pyplot as plt
plt.style.use('default')
import matplotlib.gridspec as gridspec
import matplotlib.ticker as ticker
import matplotlib.colors as colors
from matplotlib.pyplot import figure
from matplotlib.lines import Line2D

import datetime
from matplotlib.dates import DayLocator, MonthLocator, HourLocator, AutoDateLocator, DateFormatter, drange
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU, WeekdayLocator
from numpy import arange
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import FormatStrFormatter
from matplotlib.ticker import NullFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.cm as cm
import matplotlib.mlab as mlab

import glob

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [4]:
# Font / Figure Size
import matplotlib.pylab as pylab
params = {'legend.fontsize': 'xx-large',
         'axes.labelsize': 'xx-large',
         'axes.titlesize': 'xx-large',
         'xtick.labelsize': 'x-large',
         'ytick.labelsize': 'x-large'}
pylab.rcParams.update(params)
#rc('font',**{'family':'sans-serif','sans-serif':['Arial']})
#rc('text', usetex=True)

pylab.rcParams['xtick.major.pad']='12'
pylab.rcParams['ytick.major.pad']='12'
pylab.rcParams['axes.titlepad'] = 35
pylab.rcParams['axes.labelpad'] = 15

In [5]:
# Custom Colors

darkblue = '#4e79a7'
orange = '#f28e2b'
red = '#e15759'
lightblue = '#76b7b2'
green = '#59a14f'
yellow = '#edc948'
purple = '#b07aa1'
pink = '#ff9da7'
brown = '#9c755f'
gray = '#bab0ac'

# Make Legend
solar = mpatches.Patch(color=yellow, label='Solar')
wind = mpatches.Patch(color='cornflowerblue', label='Wind')
batt = mpatches.Patch(color=purple, label='Li-ion')
rfb = mpatches.Patch(color=green, label='RFB')
caes = mpatches.Patch(color=lightblue, label='CAES')
psh = mpatches.Patch(color=darkblue, label='PSH')
thermal = mpatches.Patch(color=orange, label='Thermal')
gravitational = mpatches.Patch(color=red, label='Gravitational')
metal_air = mpatches.Patch(color=gray, label='Metal-Air')
pgp = mpatches.Patch(color=pink, label='PGP')

In [8]:
%cd ..

from extract_data_sep_power import get_data_sep_power
from extract_data_one_power import get_data_one_power
from extract_data_non_sep_power import get_data_non_sep_power

/Users/annali/Library/CloudStorage/GoogleDrive-ali10@carnegiescience.edu/My Drive/clab_third_storage/Making_Figures


In [9]:
from math import log10, floor
def round_to_1(x):
    return round(x, -int(floor(log10(abs(x)))))

In [10]:
Li_ion_PGP_X = {}
Li_ion_X = {}
PGP_X = {}
X = {}

Li_ion_PGP_X['directory'] = '../Output_Data/CONUS/Three_Techs_Li-ion_PGP_X'
Li_ion_X['directory'] = '../Output_Data/CONUS/Two_Techs_Li-ion_X'
PGP_X['directory'] = '../Output_Data/CONUS/Two_Techs_PGP_X'
X['directory'] = '../Output_Data/CONUS/One_Tech_X'

Li_ion_PGP_X['techs'] = ['RFB', 'Gravitational', 'PSH', 'CAES', 'Thermal', 'Metal-Air']
Li_ion_X['techs'] = ['RFB', 'Gravitational', 'PSH', 'CAES', 'Thermal', 'Metal-Air', 'Li-ion_PGP']
PGP_X['techs'] = ['RFB', 'Gravitational', 'PSH', 'CAES', 'Thermal', 'Metal-Air']
X['techs'] = ['Li-ion', 'RFB', 'Gravitational', 'PSH', 'CAES', 'Thermal', 'Metal-Air', 'PGP']

def create_dataframe(case, keys):
    path = case['directory']
    case['data'] = {}
    case['disp_data'] = {}
    dataframe = case['disp_data']
    for tech in case['techs']:
        case['data'][tech] = {}
        dic = case['data'][tech]
        if tech == 'Metal-Air' or tech == 'Li-ion':
            
            dic = get_data_non_sep_power(path + '/' + tech, 2, 'fixed_cost', 3, 'fixed_cost', False)
            dic['to_third_tech_cost'] = [None]
            dic['to_third_tech_cap'] = [None]
            dic['to_third_tech_tot_dispatch'] = [None]
            
            dic['third_tech_energy_cost'] = dic.pop('third_tech_cost')
            dic['third_tech_energy_cap'] = dic.pop('third_tech_cap')
            dic['third_tech_energy_tot_dispatch'] = dic.pop('third_tech_tot_dispatch')
            
            dic['from_third_tech_cost'] = [None]
            dic['from_third_tech_cap'] = [None]
            dic['from_third_tech_tot_dispatch'] = [None]
            
            if tech == 'Metal-Air':
                dic['third_tech_dur'] = [100]
            if tech == 'Li-ion':
                dic['third_tech_dur'] = [4]
            
        elif tech == 'CAES' or tech == 'PGP':
            dic = get_data_sep_power(path + '/' + tech, 2, 'fixed_cost', 3, 'fixed_cost', False)
        else:
            dic = get_data_one_power(path + '/' + tech, 2, 'fixed_cost', 3, 'fixed_cost', True)
            dic['to_third_tech_cost'] = dic.pop('third_tech_power_cost')
            dic['to_third_tech_cap'] = dic.pop('third_tech_power_cap')
            dic['to_third_tech_tot_dispatch'] = dic.pop('third_tech_power_tot_dispatch')
            dic['from_third_tech_cost'] = [None]
            dic['from_third_tech_cap'] = [None]
            dic['from_third_tech_tot_dispatch'] = [None]
        data_you_want = {key: None if ((len(dic[key])==0) or (dic[key][0] is None)) else round(dic[key][0], 3) for key in keys}
        dataframe[tech] = pd.DataFrame.from_dict(data_you_want, orient='index', columns=[tech])

In [11]:
def concat_dataframe(case):
    df = case['disp_data']
    df1 = df[case['techs'][0]]
    df2 = df[case['techs'][1]]
    df3 = df[case['techs'][2]]
    df4 = df[case['techs'][3]]
    df5 = df[case['techs'][4]]
    df6 = df[case['techs'][5]]
    final_df = pd.concat([df1.set_index(df6.index), df2.set_index(df6.index), 
               df3.set_index(df6.index), df4.set_index(df6.index),
               df5.set_index(df6.index), df6.set_index(df6.index)], axis=1)
    return(final_df)

In [19]:
def save_table(dataframe):
    df = concat_dataframe(dataframe)
    doc = docx.Document('./Tables.docx')

    # add a table to the end and create a reference variable
    # extra row is so we can add the header row
    t = doc.add_table(df.shape[0]+1, df.shape[1])

    # add the header rows.
    for j in range(df.shape[-1]):
        t.cell(0,j).text = df.columns[j]

    # add the rest of the data frame
    for i in range(df.shape[0]):
        for j in range(df.shape[-1]):
            t.cell(i+1,j).text = str(df.values[i,j])

    # save the doc
    doc.save('./Tables.docx')

In [1]:
keys = ['system_cost', 
        'battery_cap',
        'to_third_tech_cap', 'third_tech_energy_cap', 'from_third_tech_cap',
        'to_PGP_cap', 'PGP_storage_cap', 'from_PGP_cap', 
        'third_tech_dur', 'PGP_dur',
        'batt_cycles', 'third_tech_cycles', 'PGP_cycles']

In [27]:
case = X

create_dataframe(case, keys)
result = concat_dataframe(case)
display(result)
result.to_csv('../Tables/One_Tech.csv', index=True)

,Li-ion,RFB,Gravitational,PSH,CAES,Thermal,Metal-Air,PGP
system_cost,0.133,0.131,0.113,0.116,0.103,0.114,0.089,0.084
battery_cap,4.381,NaN,NaN,NaN,NaN,NaN,NaN,NaN
to_third_tech_cap,NaN,0.740,0.784,0.779,0.554,0.749,NaN,NaN
third_tech_energy_cap,NaN,14.975,18.760,18.581,28.383,34.340,89.556,NaN
from_third_tech_cap,NaN,NaN,NaN,NaN,0.791,NaN,NaN,NaN
to_PGP_cap,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.198
PGP_storage_cap,NaN,NaN,NaN,NaN,NaN,NaN,NaN,368.036
from_PGP_cap,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.879
third_tech_dur,4.000,20.238,23.922,23.838,35.883,45.820,100.000,0.000
PGP_dur,0.000,NaN,NaN,NaN,0.000,NaN,0.000,418.765


In [21]:
case = Li_ion_X

create_dataframe(case, keys)
result = concat_dataframe(case)
display(result)
result.to_csv('../Tables/Two_Techs_Li-ion_X.csv', index=True)

,RFB,Gravitational,PSH,CAES,Thermal,Metal-Air,Li-ion_PGP
system_cost,0.127,0.113,0.115,0.103,0.111,0.089,0.083
battery_cap,1.136,0.558,1.005,0.096,1.009,0.730,0.714
to_third_tech_cap,0.454,0.641,0.510,0.546,0.511,NaN,NaN
third_tech_energy_cap,12.991,17.376,14.654,28.403,37.952,72.588,NaN
from_third_tech_cap,NaN,NaN,NaN,0.768,NaN,NaN,NaN
to_PGP_cap,NaN,NaN,NaN,NaN,NaN,NaN,0.179
PGP_storage_cap,NaN,NaN,NaN,NaN,NaN,NaN,371.949
from_PGP_cap,NaN,NaN,NaN,NaN,NaN,NaN,0.712
third_tech_dur,28.605,27.123,28.746,37.004,74.278,100.000,NaN
PGP_dur,NaN,NaN,NaN,0.000,NaN,0.000,522.204


In [30]:
case = PGP_X

create_dataframe(case, keys)
result = concat_dataframe(case)
display(result)
result.to_csv('../Tables/Two_Techs_PGP_X.csv', index=True)

,RFB,Gravitational,PSH,CAES,Thermal,Metal-Air
system_cost,0.084,0.084,0.084,0.083,0.084,0.083
battery_cap,NaN,NaN,NaN,NaN,NaN,NaN
to_third_tech_cap,0.029,0.163,0.043,0.095,0.000,NaN
third_tech_energy_cap,0.052,0.583,0.191,2.641,0.000,38.922
from_third_tech_cap,NaN,NaN,NaN,0.250,NaN,NaN
to_PGP_cap,0.195,0.181,0.189,0.165,0.198,0.079
PGP_storage_cap,366.558,367.673,362.759,375.212,368.036,321.453
from_PGP_cap,0.851,0.723,0.839,0.643,0.879,0.512
third_tech_dur,1.807,3.567,4.424,10.577,NaN,100.000
PGP_dur,430.835,508.812,432.587,583.293,418.765,627.715


In [12]:
case = Li_ion_PGP_X

create_dataframe(case, keys)
result = concat_dataframe(case)
display(result)
result.to_csv('../Tables/Three_Techs_Li-ion_PGP_X.csv', index=True)

,RFB,Gravitational,PSH,CAES,Thermal,Metal-Air
system_cost,0.083,0.083,0.083,0.083,0.083,0.082
battery_cap,0.714,0.714,0.714,0.114,0.714,0.717
to_third_tech_cap,0.000,0.000,0.000,0.085,0.000,NaN
third_tech_energy_cap,0.000,0.000,0.000,2.435,0.000,31.222
from_third_tech_cap,NaN,NaN,NaN,0.221,NaN,NaN
to_PGP_cap,0.179,0.179,0.179,0.165,0.179,0.091
PGP_storage_cap,371.949,371.949,371.949,375.528,371.949,346.499
from_PGP_cap,0.712,0.712,0.712,0.644,0.712,0.421
third_tech_dur,NaN,NaN,NaN,11.027,NaN,100.000
PGP_dur,522.204,522.204,522.204,582.806,522.204,823.481


OSError: Cannot save file into a non-existent directory: '../Tables'